In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hr-analytics-job-change-of-data-scientists/sample_submission.csv
/kaggle/input/hr-analytics-job-change-of-data-scientists/aug_test.csv
/kaggle/input/hr-analytics-job-change-of-data-scientists/aug_train.csv


In [2]:
import keras
import pandas as pd
import collections
from sklearn.model_selection import train_test_split
from keras.models import Sequential 
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [3]:
df_train = pd.read_csv('/kaggle/input/hr-analytics-job-change-of-data-scientists/aug_train.csv')
df_train

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19153,7386,city_173,0.878,Male,No relevent experience,no_enrollment,Graduate,Humanities,14,NaN,NaN,1,42,1.0
19154,31398,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,NaN,NaN,4,52,1.0
19155,24576,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,50-99,Pvt Ltd,4,44,0.0
19156,5756,city_65,0.802,Male,Has relevent experience,no_enrollment,High School,NaN,<1,500-999,Pvt Ltd,2,97,0.0


In [4]:
df_test = pd.read_csv('/kaggle/input/hr-analytics-job-change-of-data-scientists/aug_test.csv')

df_test

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
0,32403,city_41,0.827,Male,Has relevent experience,Full time course,Graduate,STEM,9,<10,NaN,1,21
1,9858,city_103,0.920,Female,Has relevent experience,no_enrollment,Graduate,STEM,5,NaN,Pvt Ltd,1,98
2,31806,city_21,0.624,Male,No relevent experience,no_enrollment,High School,NaN,<1,NaN,Pvt Ltd,never,15
3,27385,city_13,0.827,Male,Has relevent experience,no_enrollment,Masters,STEM,11,10/49,Pvt Ltd,1,39
4,27724,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,10000+,Pvt Ltd,>4,72
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2124,1289,city_103,0.920,Male,No relevent experience,no_enrollment,Graduate,Humanities,16,NaN,Public Sector,4,15
2125,195,city_136,0.897,Male,Has relevent experience,no_enrollment,Masters,STEM,18,NaN,NaN,2,30
2126,31762,city_100,0.887,Male,No relevent experience,no_enrollment,Primary School,NaN,3,NaN,Pvt Ltd,never,18
2127,7873,city_102,0.804,Male,Has relevent experience,Full time course,High School,NaN,7,100-500,Public Sector,1,84


In [5]:
df_test.isnull().sum().sum()

2204

In [6]:
def ready_dataset(x):
    a = x.isnull().sum().tolist()
    for i in a:
        if i != 0:
            if x[x.columns[a.index(i)]].dtype == 'object' or x[x.columns[a.index(i)]].dtype == 'bool':
                x[x.columns[a.index(i)]].fillna(x[x.columns[a.index(i)]].mode()[0], inplace = True)
            elif x[x.columns[a.index(i)]].dtype == 'int64'or x[x.columns[a.index(i)]].dtype == 'float64':
                x[df.columns[a.index(i)]].fillna(x[df.columns[a.index(i)]].mean(), inplace = True)
    
    encoder_dict = collections.defaultdict(LabelEncoder)

    x = x.apply(lambda y: encoder_dict[y.name].fit_transform(y))
    
    return x    

In [7]:
df_train = ready_dataset(df_train)
df_test = ready_dataset(df_test)
# df

In [8]:
x_train = df_train.iloc[:, 2:13].values
y_train = df_train.iloc[:,13].values

In [9]:
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.4, random_state = 0)
x_test = df_test.iloc[:, 2:13].values

In [10]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)    
x_test = sc.transform(x_test)

In [11]:
def mymodel():
    model = Sequential()
    
    model.add(Dense(100, activation = 'relu', input_shape = (11,)))
    model.add(Dense(80, activation = 'relu'))
    model.add(Dense(20, activation = 'relu'))
    model.add(Dense(1, activation = 'sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

mod = mymodel()
mod.fit(x_train, y_train, epochs=100, verbose=2)

Epoch 1/100
599/599 - 1s - loss: 0.4978 - accuracy: 0.7616
Epoch 2/100
599/599 - 1s - loss: 0.4828 - accuracy: 0.7704
Epoch 3/100
599/599 - 1s - loss: 0.4779 - accuracy: 0.7725
Epoch 4/100
599/599 - 1s - loss: 0.4750 - accuracy: 0.7751
Epoch 5/100
599/599 - 1s - loss: 0.4712 - accuracy: 0.7778
Epoch 6/100
599/599 - 1s - loss: 0.4692 - accuracy: 0.7789
Epoch 7/100
599/599 - 1s - loss: 0.4652 - accuracy: 0.7788
Epoch 8/100
599/599 - 1s - loss: 0.4631 - accuracy: 0.7807
Epoch 9/100
599/599 - 1s - loss: 0.4615 - accuracy: 0.7825
Epoch 10/100
599/599 - 1s - loss: 0.4594 - accuracy: 0.7821
Epoch 11/100
599/599 - 1s - loss: 0.4576 - accuracy: 0.7855
Epoch 12/100
599/599 - 1s - loss: 0.4565 - accuracy: 0.7839
Epoch 13/100
599/599 - 1s - loss: 0.4548 - accuracy: 0.7852
Epoch 14/100
599/599 - 1s - loss: 0.4517 - accuracy: 0.7867
Epoch 15/100
599/599 - 1s - loss: 0.4506 - accuracy: 0.7869
Epoch 16/100
599/599 - 1s - loss: 0.4497 - accuracy: 0.7877
Epoch 17/100
599/599 - 1s - loss: 0.4470 - accura

In [12]:
pred = mod.predict(x_test)
pred

array([[4.3100804e-02],
       [1.5321374e-04],
       [3.6585066e-01],
       ...,
       [6.6681569e-06],
       [7.6392293e-04],
       [2.4828610e-01]], dtype=float32)

In [13]:
ans = []
for i in pred:
    if i > 0.5:
        ans.append(1)
    else:
        ans.append(0)
ans

[0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [14]:
d = pd.read_csv('/kaggle/input/hr-analytics-job-change-of-data-scientists/sample_submission.csv')
# d

In [15]:
d['target'] = ans
d.to_csv('Submission.csv', index = False)